# 12. 使用FP-growth算法来高效发现频繁集

FP-growth 算法基于Apriori构建，但在完成相同任务时采用了一些不同的技术。这里的任务是将数据集存在一个称为FP树的结构之后发现频繁项集或频繁项对，即常在一块出现的元素项的集合FP树。这种做法使得算法的执行速度要快于Apriori。

该算法能更为高效的发现频繁项集，但不能用于发现关联规则。FP-growth算法只扫描数据两次，它发现频繁项集的基本过程如下：

1. 构建FP树
2. 从FP树中挖掘频繁项集

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 12.1. 创建FP树的数据结构

In [3]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    # 增加count的值
    def inc(self, numOccur):
        self.count += numOccur
    
    # 用于打印出树，便于调试
    def disp(self, ind=1):
        print(" "*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
# 测试代码
rootNode = treeNode('pyramid', 9, None)
rootNode.children['eye'] = treeNode('eye', 13, None)
rootNode.disp()

  pyramid   9
   eye   13


## 12.2. 构建FP树

除了FP树之外，还需要一个头指针来指向给定类型的第一个实例。利用头指针表可以快速访问FP树中的一个给定类型的所有元素。
![](FP-tree.png)
这里使用一个字典作为数据结构，来保存头部指针，除了存放头部指针，还保存FP树中每类元素的总数

第一遍遍历数据集会获得每个元素项的出现频率。接下来去掉不满足最小支持度的元素项，再下一步构建FP树。

假设有以下交易：

| 交易id | 交易项 |
|:-----:|:-----:|
| 1 | r,z,h,j,p |
| 2 | z,y,x,w,v,u,t,s |
| 3 | z |
| 4 | r,x,n,o,s |
| 5 | y,r,x,z,q,t,p |
| 6 | y,z,x,e,q,s,t,m |

首先遍历一遍数据集，生成头指针表，并排序，把小于最小支持度的元素丢弃

In [34]:
# 生成头部指针表，注意：这里并没有添加头部指针
def genHeadPointerList(dataSet, minsup = 2):
    headdict = {}
    retdict = {}
    for trans in dataSet:
        for i in trans:
            headdict[i] = headdict.get(i,0) + 1
    
    for key,val in headdict.items():
        if val >= minsup:
            retdict[key] = val
            
    return retdict

In [29]:
def loadData(file):
    dataSet = []
    with open(file, 'r') as f:
        for line in f.readlines():
            dataSet.append(line.strip().split(','))
    return dataSet

In [30]:
dataSet = loadData('test')

In [35]:
genHeadPointerList(dataSet, 3)

{'r': 3, 's': 3, 't': 3, 'x': 3, 'y': 3, 'z': 5}

In [38]:
[v[0] for v in sorted(genHeadPointerList(dataSet, 3).items(), key = lambda kv:kv[1], reverse=True)]

['z', 't', 's', 'r', 'y', 'x']

将得到的结果进行排序，除了z之外其他元素的个数相同，这里不妨假设排序的结果为： z,x,y,s,r,t,于是，根据这个结果我们得到了新的交易列表

| 交易id | 交易项 |
|:-----:|:-----:|
| 1 | z,r |
| 2 | z,x,y,s,t |
| 3 | z |
| 4 | x,s,r |
| 5 | z,x,y,r,t |
| 6 | y,x,y,s,t |

接下来我们就可以构建FP树了。下面这张图展示了根据上面表中的数据构建FP树的过程。
![](FP-Tree-gen.png)
依次遍历每一个交易项，从空集开始，向其中不断添加，如果树中已存在现有元素则增加元素的值，否则添加一个新的分支。

In [60]:
def createTree(dataSet, minSup = 1):
    headerTable = genHeadPointerList(dataSet, minSup)
    freqItemSet = set(headerTable)
    
    # 如果没有元素满足要求，则返回None
    if len(freqItemSet) == 0:
        return None, None
    
    # 重新组织字典，为其添加头部指针，初始化为None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
        
    retTree = treeNode("Null Set", 1, None)
    
    # 根据排序后的结果生成新的交易列表，同时生成树
    for trans in dataSet:
        localD = {}
        for item in trans:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0: 
            # 按照出现频率排序，生成新的交易列表
            orderedItems = [v[0] for v in sorted(localD.items(), key = lambda kv:kv[1], reverse=True)]
            print(orderedItems)
            updateTree(orderedItems, retTree, headerTable)
                
    return retTree, headerTable


def updateTree(items, inTree, headerTable):
#     print(items[0])
    if items[0] in inTree.children:
        # 计数增加1
        inTree.children[items[0]].inc(1)
    else:
        # 当前元素没有在树中，建立新子树
        inTree.children[items[0]] = treeNode(items[0], 1, inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
            
    if len(items) > 1:
        updateTree(items[1:], inTree.children[items[0]], headerTable)
        
        
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
    
        

In [61]:
myTree, headerTable = createTree(dataSet, 3)
myTree.disp()

['z', 'r']
['z', 's', 't', 'y', 'x']
['z']
['r', 's', 'x']
['z', 'r', 't', 'y']
['z', 's', 't', 'y', 'x']
  Null Set   1
   r   1
    s   1
     x   1
   z   5
    r   2
     t   1
      y   1
    s   2
     t   2
      y   2
       x   2
